In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from valuation.Manager import Valuation
from utilities.utils import fullpath, get_datestr
from datetime import datetime, timedelta

In [3]:
value.last_working_day

Timestamp('2025-03-31 00:00:00')

In [8]:
value = Valuation(20250331, "cw")
df_weights = value.valuation_quarterly()

         Date      AC.PA  ACA.PA       AI.PA      AIR.PA      BN.PA  \
0  2024-12-02  43.459999  12.560  159.539993  150.520004  64.839996   
1  2024-12-03  43.759998  12.510  160.020004  152.080002  64.800003   
2  2024-12-04  44.000000  12.635  161.160004  154.220001  64.739998   
3  2024-12-05  46.099998  12.925  161.660004  153.539993  64.980003   
4  2024-12-06  46.020000  13.035  162.059998  155.160004  64.860001   
..        ...        ...     ...         ...         ...        ...   
78 2025-03-25  44.910000  17.295  178.720001  172.320007  71.059998   
79 2025-03-26  44.820000  17.135  176.740005  170.539993  70.540001   
80 2025-03-27  44.220001  17.090  176.440002  168.699997  70.820000   
81 2025-03-28  42.790001  16.945  176.320007  166.259995  70.820000   
82 2025-03-31  41.889999  16.775  175.139999  162.779999  70.839996   

       BNP.PA     BVI.PA   CA.PA      CAP.PA  ...      SAF.PA      SAN.PA  \
0   55.939999  28.820000  13.710  150.850006  ...  221.800003   91.820

In [ ]:
all_quarters = pd.read_csv("data/quarters.txt",names=["Dates"],skiprows=1)
all_quarters.Dates = pd.to_datetime(all_quarters.Dates, format="%Y%m%d").dt.date
all_quarters

In [ ]:
# create daily valuation
today = datetime.today()
t1 = timedelta(days=1)

current_quarter = fullpath(value.folder_path_quarterly, today.strftime("%Y%m%d")+".csv")
df_num_shares = pd.read_csv(fullpath( value.folder_path_quarterly, get_datestr(value.previous_quarter)+".csv" ))

symbol = df_num_shares["Symbol"].to_list()

market_data = yf.download(symbol, start=today-t1*7, end=today, interval="1d", group_by="column") 
market_data = market_data.xs("Close",level=0, axis=1)

# check market data and symbol alignment
if all(market_data.columns == symbol):
    prices = np.matrix( market_data.values )
    num_shares = np.matrix(df_num_shares["NumShares"]).transpose()

    # check inner size of matrix prior to multiplication
    if prices.shape[1] == num_shares.shape[0]:
        portfolio_value = prices @ num_shares
        df_portfolio_value = pd.DataFrame(portfolio_value, columns=["Portfolio"])
        df_portfolio_value.index = market_data.index.date

        # publish daily portfolio value
        for d in range(len(df_portfolio_value)):
            current_dte = df_portfolio_value.iloc[d,:].name.strftime("%Y%m%d")
            fname = fullpath("data","valuation","daily","cw",current_dte+".csv")
            df_portfolio_value.iloc[[d]].to_csv(fname, index=False,header=False)



In [ ]:
df_portfolio_value